# Data Preprocessing Template Lite
1. Import lib
1. Import data
1. EDA - Exploratory Data Analysis
1. Separate target and features
1. Split train set and test set (80/20)
1. Cleaning
    1. Replace missing val
    1. Standardize numerical features + One hot encode categorical variables
    1. Encode labels of y
1. Apply pre processing to test set


## Step 1 - Import libraries

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

## Step 2 - Import dataset

In [20]:
df = pd.read_csv("assets/ML/Data.csv")
df.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


## Step 3 - EDA - Exploratory Data Analysis

In [21]:
print(df.shape)                       # (#rows, #columns)
print(df.describe(include="all"))

# Vérifier que le count est identique pour toutes les colonnes
# Attention on peut avoir le même nb de valeurs qui manquent partout
# Faut correler avec df.shape

# Here, there are tons of options 
  # df.value_counts()
  # df = df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])
  # sns.catplot(data = df, x="Sex", y="Survived", kind="bar")
  # print(df[df['Embarked'].isna()]) # print lines with NaN
  # df.isna().sum() / len(df) * 100 # print nb of Nana as %

(10, 4)
       Country        Age        Salary Purchased
count       10   9.000000      9.000000        10
unique       3        NaN           NaN         2
top     France        NaN           NaN        No
freq         4        NaN           NaN         5
mean       NaN  38.777778  63777.777778       NaN
std        NaN   7.693793  12265.579662       NaN
min        NaN  27.000000  48000.000000       NaN
25%        NaN  35.000000  54000.000000       NaN
50%        NaN  38.000000  61000.000000       NaN
75%        NaN  44.000000  72000.000000       NaN
max        NaN  50.000000  83000.000000       NaN


### Voir par exemple que ici :
* Y a des valeurs manquantes pour toutes les colonnes (count pas identique partout)
* 3 catégories uniques de pays
* target (purchased) de type Yes/No

## Step 3 - Separate Target from feature variables

In [22]:
features_list = ["Country", "Age", "Salary"]
X = df.loc[:, features_list]
y = df.loc[:, "Purchased"]
# print(X.head())
# print(y.head())

## Step 4 - Train / Test split

In [23]:
# Usually : 80% training and 20% testing  
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=0,       # donne une valeur pour être sûr d'avoir toujours le même comportement random
                                                    stratify=y)           # Allows you to stratify our sample. 
                                                                          # We will have the same proportion of categories in test and train set

## Step 5 - Cleaning

1. Remplace missing values - SimpleImputer avec strategy='mean' pour valeurs numériques ou strategy='mode' si il manque des données catégorielles
1. Standardize numerical features + One hot encode categorical variables
1. If y is discrete => Encode labels of y

### 5.1 - Replace missing values

In [24]:
# Missing values - SimpleImputer
# Si il avait manqué un nom de pays, on aurait pu utiliser une strategie "mode" pour remplacer par le pays le plus fréquent
print(X_train)
imputer = SimpleImputer(strategy="mean")  # Instanciate class of SimpleImputer with strategy of mean
#X_train = X_train.copy()                 # ! Copy dataset to avoid caveats of assign a copy of a slice of a DataFrame
                                          # Semble plus nécessaire en 2023
                                          # More info here https://towardsdatascience.com/explaining-the-settingwithcopywarning-in-pandas-ebc19d799d25

X_train.iloc[:,[1,2]] = imputer.fit_transform(X_train.iloc[:,[1,2]]) # Fit and transform columns where there are missing values
print() 
print(X_train) 

   Country   Age   Salary
0   France  44.0  72000.0
4  Germany  40.0      NaN
6    Spain   NaN  52000.0
9   France  37.0  67000.0
3    Spain  38.0  61000.0
1    Spain  27.0  48000.0
2  Germany  30.0  54000.0
5   France  35.0  58000.0

   Country        Age        Salary
0   France  44.000000  72000.000000
4  Germany  40.000000  58857.142857
6    Spain  35.857143  52000.000000
9   France  37.000000  67000.000000
3    Spain  38.000000  61000.000000
1    Spain  27.000000  48000.000000
2  Germany  30.000000  54000.000000
5   France  35.000000  58000.000000


### 5.2 - Standardize numerical features + One hot encode categorical variables

In [25]:
# Standardizing numeric features and encoding categorical features

numeric_features = [1, 2]                             # On crée une liste avec les indices des colonnes qui contiennent des valeurs numériques
                                                      # Age et Salaires sont dans les colonnes 1 et 2
numeric_transformer = StandardScaler()                # On précise le type de transformer qu'on veut utiliser pour les val numériques

categorical_features = [0]                            # On crée une liste avec les indices des colonnes qui contiennent des valeurs catégorielles
                                                      # Les pays sont dans la colonne d'indice 0 (la première)
categorical_transformer = OneHotEncoder(drop='first') # Pour virer l'Angleterre comme dans l'exemple ci-dessus on aurait mis drop="last" mais ça n'existe pas
                                                      # Il faut garder le drop first car sinon on a 2 colonnes 
                                                      # qui sont 100% corrélées est c'est pas bon pour le modèle

featureencoder = ColumnTransformer(                   # ColumnTransformer provient du module compose
    transformers=[                                    # On passe une liste de transformers à qui ont donne un nom (cat, num...)
        ('cat', categorical_transformer, categorical_features),   
        ('num', numeric_transformer, numeric_features)
        ]
    )

# La variable featureencoder est un object de type ColumnTransformer
# Elle contient la "recette" pour transformer chacune des colonnes
# Sur les colonnes 1 et 2 qui sont de type numérique appliquer StandarScaler
# Sur la colonne 0 qui est de type catégorielle, appliquer OneHotEncoder
# ... 
# L'énorme avantage de procéder comme ça c'est que 
#     si on veut tester un ou ajouter des transformers sur des colonnes on peut le faire en modifiant le code à un seul endroit
#     on est sûr d'appliquer la même "recette" plus tard à nos données de test (X_test)

X_train = featureencoder.fit_transform(X_train)
print(X_train[:5])  # print first 5 rows (not using iloc since now X_train became a numpy array)
                    # ! X_train became a numpy array

# On a 4 colonnes à l'affichage car il y a 2 pays, age et salary
# France    => 0 et 0
# Allemagne => 1 et 0
# Spain     => 0 et 1


[[ 0.00000000e+00  0.00000000e+00  1.61706195e+00  1.78674463e+00]
 [ 1.00000000e+00  0.00000000e+00  8.22715727e-01  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00 -1.41104234e-15 -9.32214592e-01]
 [ 0.00000000e+00  0.00000000e+00  2.26956063e-01  1.10700483e+00]
 [ 0.00000000e+00  1.00000000e+00  4.25542617e-01  2.91317060e-01]]


### 5.3 - Encode labels of y

In [26]:
# Encoding labels
# Replace "yes" / "no" by `0` and `1` which can be interpreted by a computer. 

print(y_train)
labelencoder = LabelEncoder()                       # LabelEncoder provient de sklearn.preprocessing
                                                    # Va transformer les Yes, No en 0, 1
                                                    # Si on avait eu Riri, Fifi, Loulou en lables différents
                                                    # il aurait codé en 0, 1 et 2
y_train = labelencoder.fit_transform(y_train)
print(y_train[:5])                                  # print first 5 rows (not using iloc since now y_train became a numpy array)
                                                    # ! y_train became a numpy array

0     No
4    Yes
6     No
9    Yes
3     No
1    Yes
2     No
5    Yes
Name: Purchased, dtype: object
[0 1 0 1 0]


## Step X - Training

## Step 6 - Apply preprocessing to test set

In [27]:
# 5.1 - Missing values
# print(X_test)
# X_test = X_test.copy()                                          # !Z
                                                                  # Semble plus nécessaire en 2023
X_test.iloc[:,[1,2]] = imputer.transform(X_test.iloc[:,[1,2]])    # On réutilise l'objet imputer
# print(X_test) 

# 5.2 - Encoding categorical features and standardizing numeric features
X_test = featureencoder.transform(X_test)       # On réutilise la "recette" contenue dans l'objet featureencoder
                                                # On est sûr et certains de traiter les données de test de la même façon que les données de training  
# print(X_test)

# 5.3 - Encoding labels
# print(y_test)
y_test = labelencoder.transform(y_test)         # On réutilise l'objet labelencoder
# print(y_test)

## Step 7 - Predict and evaluate